In [25]:
import pyro
import torch
import numpy as np
import pyro.poutine as poutine
import pyro.distributions as dist

pyro.set_rng_seed(101)

def walk_model():
    import pyro
    import torch

    start = pyro.sample("start", pyro.distributions.Uniform(0, 3), is_cont=False)
    t = 0
    position = start
    distance = torch.tensor(0.0)
    while position > 0 and position < 10:
        step = pyro.sample(f"step_{t}", pyro.distributions.Uniform(-1, 1), is_cont=False)
        distance = distance + torch.abs(step)
        position = position + step
        t = t + 1
    pyro.sample("obs", pyro.distributions.Normal(1.1, 0.1), obs=distance, is_cont=False)
    return start.item()

def make_log_joint(model):
    def _log_joint(cond_data, *args, **kwargs):
        conditioned_model = poutine.condition(model, data=cond_data)
        trace = poutine.trace(conditioned_model).get_trace(*args, **kwargs)
        # print(dict(trace))
        return trace.log_prob_sum(), dict(trace.nodes)
    return _log_joint
scale_log_joint = make_log_joint(walk_model)

value, trace = scale_log_joint({"start": torch.tensor(0.5), "step_0": torch.tensor(-0.9), "step_1": torch.tensor(-0.9), "obs": torch.tensor(1.3)})

from scipy.stats import norm

for site_name in trace:
    print(site_name, trace[site_name])
dic = {site_name: trace[site_name]['value'] for site_name in trace if site_name not in ["_INPUT", "_RETURN", "obs"]}
print(list(dic.items())[0])
print(tuple(dic.keys()))
print(len(dic.keys()))
print({site_name: trace[site_name]['value'] for site_name in trace if site_name not in ["_INPUT", "_RETURN", "obs"]})

print(np.log(norm(1.1, 0.1).pdf(1.3) * 1/2 * 1/2 * 1/3))
print(value)

_INPUT {'name': '_INPUT', 'type': 'args', 'args': (), 'kwargs': {}}
start {'type': 'sample', 'name': 'start', 'fn': Uniform(low: 0.0, high: 3.0), 'is_observed': True, 'args': (), 'kwargs': {}, 'value': tensor(0.5000), 'infer': {}, 'scale': 1.0, 'mask': None, 'cond_indep_stack': (), 'done': True, 'stop': False, 'continuation': None, 'is_cont': False, 'log_prob_sum': tensor(-1.0986)}
step_0 {'type': 'sample', 'name': 'step_0', 'fn': Uniform(low: -1.0, high: 1.0), 'is_observed': True, 'args': (), 'kwargs': {}, 'value': tensor(-0.4000), 'infer': {}, 'scale': 1.0, 'mask': None, 'cond_indep_stack': (), 'done': True, 'stop': False, 'continuation': None, 'is_cont': False, 'log_prob_sum': tensor(-0.6931)}
step_1 {'type': 'sample', 'name': 'step_1', 'fn': Uniform(low: -1.0, high: 1.0), 'is_observed': True, 'args': (), 'kwargs': {}, 'value': tensor(-0.9000), 'infer': {}, 'scale': 1.0, 'mask': None, 'cond_indep_stack': (), 'done': True, 'stop': False, 'continuation': None, 'is_cont': False, 'log_p

In [35]:
pyro.sample("is_test", torch.distributions.Laplace(torch.zeros(10), torch.ones(10)).sample)
for i in range(3, 10):
    print(i)

3
4
5
6
7
8
9


In [36]:
def _potential_fn(self, params):
        params_constrained = {k: self.transforms[k].inv(v) for k, v in params.items()}
        cond_model = poutine.condition(self.model, params_constrained)
        model_trace = poutine.trace(cond_model).get_trace(
            *self.model_args, **self.model_kwargs
        )
        log_joint = self.trace_prob_evaluator.log_prob(model_trace)
        for name, t in self.transforms.items():
            log_joint = log_joint - torch.sum(
                t.log_abs_det_jacobian(params_constrained[name], params[name])
            )
        return -log_joint



In [39]:
print(dic)
dic.pop('start')
print(dic)

{'step_0': tensor(-0.4000), 'step_1': tensor(-0.9000)}


KeyError: 'start'

In [40]:
torch.isfinite(torch.tensor(14.9))

tensor(True)